In [17]:
import numpy as np
import matplotlib.pyplot as plt


In [18]:
particle_id = []
part = 8
npol = 40
n_chain = int(part*npol)
bead = 2
beadpol = int(bead*npol)
for i in range(0,part):
    particle_id.append(int((i+1)+(i+1)*beadpol))
print(particle_id)

[81, 162, 243, 324, 405, 486, 567, 648]


In [19]:
global bead1; bead1 = []
s = True
for i in range(1,int(bead*npol*part+part)+1):
    if i%2 !=0 and s:
        if i%81 != 0:
            bead1.append(i)
        else:
            s = not s
    elif i%2 == 0 and not s:
        if i%81 != 0:
            bead1.append(i)
        else:
            s = not s
global bead2; bead2 = []
for i in range(0,len(bead1)):
    bead2.append(bead1[i]+1)
bead1 = np.array(bead1)
bead2 = np.array(bead2)        

In [20]:
def find_pair(p1):
    global bead1,bead2;
    if p1 in bead1:
        i = np.where(bead1 == p1) 
        return(bead2[i[0][0]])
    elif p1 in bead2:
        i = np.where(bead2 == p1) 
        return(bead1[i[0][0]])

In [32]:
def bead_index(p):
    global bead1, bead2;
    if p in bead1:
        i = np.where(bead1 == p)
        return(i[0][0])
    if p in bead2:
        i = np.where(bead2 == p)
        return(i[0][0])

In [38]:
filename = "dump.stat"
time_step = []
line_counter = 0
n_pairs = 0
with open(filename) as file1:
    for line in file1:
        line_counter += 1
        if line_counter == 2:
            time_step.append(int(line))
        if line_counter == 4:
            n_pairs = int(line)
        if line_counter == n_pairs+9:
            line_counter = 0

line_counter = 0
#ch_type = np.zeros((len(time_step),n_chain),dtype = int)
bead1_ener = np.zeros((len(time_step),n_chain))
bead2_ener = np.zeros((len(time_step),n_chain))
n_pairs = 0
itera = 0
ch = 0
with open(filename) as file1:
    for line in file1:
        line_counter += 1
        if line_counter == 4:
            n_pairs = int(line)
            p1 = np.zeros(n_pairs)
            p2 = np.zeros(n_pairs)
            dist = np.zeros(n_pairs)
            ener = np.zeros(n_pairs)
            it = 0
        if line_counter >=10 and line_counter <= n_pairs+9:
            t_list = [float(i) for i in line.split()]
            p1[it] = int(t_list[0])
            p2[it] = int(t_list[1])
            dist[it] = t_list[2]
            ener[it] = t_list[3]  
            it += 1                                   
        if line_counter == n_pairs+9:
            line_counter = 0
            for i in range(0,len(p1)):
                if p1[i] in particle_id:
                    if ener[i] < 0:
                        #ch = int((p2[i] - (p2[i]/beadpol) - 1)/bead)
                        for j in range(0,len(p2)):
                            if p1[j] == find_pair(p2[i]):
                                if p2[j] in particle_id and p2[j] != p1[i] and ener[j] < 0:
                                    #ch_type[itera,ch] == 1
                                    bead1_ener[itera,bead_index(p2[i])] = ener[i]
                                    bead2_ener[itera,bead_index(p1[j])] = ener[j]
                                    break;
                            elif p2[j] == find_pair(p2[i]):
                                if p1[j] in particle_id and p1[j] != p1[i] and ener[j] < 0:
                                    #ch_type[itera,ch] == 1
                                    bead1_ener[itera,bead_index(p2[i])] = ener[i]
                                    bead2_ener[itera,bead_index(p2[j])] = ener[j]
                                    break;
                elif p2[i] in particle_id:
                    if ener[i] < 0:
                        #ch = int((p1[i] - (p1[i]/beadpol) - 1)/bead)
                        for j in range(0,len(p1)):
                            if p1[j] == find_pair(p1[i]):
                                if p2[j] in particle_id and p2[j] != p2[i] and ener[j] < 0:
                                    #ch_type[itera,ch] == 1
                                    bead1_ener[itera,bead_index(p1[i])] = ener[i]
                                    bead2_ener[itera,bead_index(p1[j])] = ener[j]
                                    break;
                            elif p2[j] == find_pair(p1[i]):
                                if p1[j] in particle_id and p1[j] != p2[i] and ener[j] < 0:
                                    #ch_type[itera,ch] == 1
                                    bead1_ener[itera,bead_index(p1[i])] = ener[i]
                                    bead2_ener[itera,bead_index(p2[j])] = ener[j]
                                    break;
            itera += 1  

          

In [ ]:
#file = open("file1.txt", "w+")
#content = str(ch_type)
#file.write(content)
#file.close()

In [ ]:
time_step = np.array(time_step).transpose()

In [ ]:
np.savetxt("bead1.txt",bead1_ener,fmt = "%0.3f")
np.savetxt("bead2.txt",bead2_ener,fmt = "%0.3f")
np.savetxt("time_step.txt",time_step,fmt = "%d")